In [12]:
from pypokerengine.engine.hand_evaluator import HandEvaluator
from pypokerengine.players import BasePokerPlayer
from pypokerengine.utils.card_utils import _pick_unused_card, _fill_community_card, gen_cards


# Estimate the ratio of winning games given the current state of the game
def estimate_win_rate(nb_simulation, nb_player, hole_card, community_card=None):
    if not community_card: community_card = []

    # Make lists of Card objects out of the list of cards
    community_card = gen_cards(community_card)
    hole_card = gen_cards(hole_card)

    # Estimate the win count by doing a Monte Carlo simulation
    win_count = sum([montecarlo_simulation(nb_player, hole_card, community_card) for _ in range(nb_simulation)])
    return 1.0 * win_count / nb_simulation


def montecarlo_simulation(nb_player, hole_card, community_card):
    # Do a Monte Carlo simulation given the current state of the game by evaluating the hands
    community_card = _fill_community_card(community_card, used_card=hole_card + community_card)
    unused_cards = _pick_unused_card((nb_player - 1) * 2, hole_card + community_card)
    opponents_hole = [unused_cards[2 * i:2 * i + 2] for i in range(nb_player - 1)]
    opponents_score = [HandEvaluator.eval_hand(hole, community_card) for hole in opponents_hole]
    my_score = HandEvaluator.eval_hand(hole_card, community_card)
    return 1 if my_score >= max(opponents_score) else 0


class DataBloggerBot(BasePokerPlayer):
    def __init__(self):
        super().__init__()
        self.wins = 0
        self.losses = 0

    def declare_action(self, valid_actions, hole_card, round_state):
        # Estimate the win rate
        win_rate = estimate_win_rate(100, self.num_players, hole_card, round_state['community_card'])

        # Check whether it is possible to call
        can_call = len([item for item in valid_actions if item['action'] == 'call']) > 0
        if can_call:
            # If so, compute the amount that needs to be called
            call_amount = [item for item in valid_actions if item['action'] == 'call'][0]['amount']
        else:
            call_amount = 0

        amount = None

        # If the win rate is large enough, then raise
        if win_rate > 0.5:
            raise_amount_options = [item for item in valid_actions if item['action'] == 'raise'][0]['amount']
            if win_rate > 0.85:
                # If it is extremely likely to win, then raise as much as possible
                action = 'raise'
                amount = raise_amount_options['max']
            else:
                # If there is a chance to win, then call
                action = 'call'
        else:
            action = 'call' if can_call and call_amount == 0 else 'fold'

        # Set the amount
        if amount is None:
            items = [item for item in valid_actions if item['action'] == action]
            amount = items[0]['amount']

        return action, amount

    def receive_game_start_message(self, game_info):
        self.num_players = game_info['player_num']

    def receive_round_start_message(self, round_count, hole_card, seats):
        pass

    def receive_street_start_message(self, street, round_state):
        pass

    def receive_game_update_message(self, action, round_state):
        pass

    def receive_round_result_message(self, winners, hand_info, round_state):
        is_winner = self.uuid in [item['uuid'] for item in winners]
        self.wins += int(is_winner)
        self.losses += int(not is_winner)


def setup_ai():
    return DataBloggerBot()

In [13]:
from pypokerengine.players import BasePokerPlayer
import numpy as np
from sklearn.neural_network import MLPRegressor


class CallBot(BasePokerPlayer):
    def declare_action(self, valid_actions, hole_card, round_state):
        actions = [item for item in valid_actions if item['action'] in ['call']]
        return list(np.random.choice(actions).values())

    def receive_game_start_message(self, game_info):
        pass

    def receive_round_start_message(self, round_count, hole_card, seats):
        pass

    def receive_street_start_message(self, street, round_state):
        pass

    def receive_game_update_message(self, action, round_state):
        pass

    def receive_round_result_message(self, winners, hand_info, round_state):
        pass


def setup_ai():
    return CallBot()

In [14]:
from pypokerengine.api.game import start_poker, setup_config
import numpy as np

if __name__ == '__main__':
    blogger_bot = DataBloggerBot()

    # The stack log contains the stacks of the Data Blogger bot after each game (the initial stack is 100)
    stack_log = []
    for round in range(100):
        p1, p2 = blogger_bot, CallBot()

        config = setup_config(max_round=5, initial_stack=100, small_blind_amount=5)
        config.register_player(name="p1", algorithm=p1)
        config.register_player(name="p2", algorithm=p2)
        game_result = start_poker(config, verbose=0)

        stack_log.append([player['stack'] for player in game_result['players'] if player['uuid'] == blogger_bot.uuid])
        print('Avg. stack:', '%d' % (int(np.mean(stack_log))))

Avg. stack: 200
Avg. stack: 167
Avg. stack: 111
Avg. stack: 107
Avg. stack: 126
Avg. stack: 117
Avg. stack: 129
Avg. stack: 134
Avg. stack: 133
Avg. stack: 126
Avg. stack: 130
Avg. stack: 130
Avg. stack: 124
Avg. stack: 115
Avg. stack: 111
Avg. stack: 110
Avg. stack: 109
Avg. stack: 109
Avg. stack: 106
Avg. stack: 111
Avg. stack: 109
Avg. stack: 113
Avg. stack: 115
Avg. stack: 114
Avg. stack: 113
Avg. stack: 117
Avg. stack: 118
Avg. stack: 117
Avg. stack: 120
Avg. stack: 119
Avg. stack: 120
Avg. stack: 118
Avg. stack: 119
Avg. stack: 121
Avg. stack: 118
Avg. stack: 116
Avg. stack: 115
Avg. stack: 117
Avg. stack: 116
Avg. stack: 116
Avg. stack: 117
Avg. stack: 119
Avg. stack: 118
Avg. stack: 120
Avg. stack: 119
Avg. stack: 118
Avg. stack: 120
Avg. stack: 118
Avg. stack: 120
Avg. stack: 120
Avg. stack: 120
Avg. stack: 121
Avg. stack: 121
Avg. stack: 122
Avg. stack: 121
Avg. stack: 120
Avg. stack: 119
Avg. stack: 120
Avg. stack: 119
Avg. stack: 119
Avg. stack: 118
Avg. stack: 117
Avg. sta